In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import keras
from keras.models import load_model

In [ ]:
model = load_model("model.h5")
label = np.load("labels.npy")

In [ ]:
holistic = mp.solutions.holistic
hands = mp.solutions.hands
holis = holistic.Holistic()
drawing = mp.solutions.drawing_utils

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    lst = []
    _, frame = cap.read()
    
    frame = cv2.flip(frame, 1)
    
    res = holis.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    if res.face_landmarks:
        for i in res.face_landmarks.landmark:
            lst.append(i.x - res.face_landmarks.landmark[1].x)
            lst.append(i.y - res.face_landmarks.landmark[1].y)
            
        if res.left_hand_landmarks:
            for i in res.left_hand_landmarks.landmark:
                lst.append(i.x - res.left_hand_landmarks.landmark[8].x)
                lst.append(i.y - res.left_hand_landmarks.landmark[8].y)
                
        else:
            for i in range(42):
                lst.append(0.0)
                
        if res.right_hand_landmarks:
            for i in res.right_hand_landmarks.landmark:
                lst.append(i.x - res.right_hand_landmarks.landmark[8].x)
                lst.append(i.y - res.right_hand_landmarks.landmark[8].y)
                
        else:
            for i in range(42):
                lst.append(0.0)
                
        lst = np.array(lst).reshape(1,-1)
        pred = label[np.argmax(model.predict(lst))]
        print(pred)
        cv2.putText(frame, pred, (50,50), cv2.FONT_ITALIC, 1, (255,0,0), 2)
    
    drawing.draw_landmarks(frame, res.face_landmarks, holistic.FACEMESH_CONTOURS)
    drawing.draw_landmarks(frame, res.right_hand_landmarks, hands.HAND_CONNECTIONS)
    drawing.draw_landmarks(frame, res.left_hand_landmarks, hands.HAND_CONNECTIONS)
    
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()